<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex08/patrick_ferreira/ex08/patrick_ferreira_175480.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from operator import itemgetter

nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

#  Exercício: Modelo de Linguagem com auto-atenção (versão eficiente)

Este exercício é similar ao da aula 5, mas iremos agora treinar *eficientemente* uma rede neural com uma ou mais camadas de auto-atenção para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Para tanto, deve-se implementar:
1. A máscara causal de atenção. Ela possibilitará que, durante o treinamento, com apenas uma forward+backward pass na rede, tenhamos as losses para todos os tokens de entrada (slide 117).
2. A máscara de PADs, que permite que usemos sequencias de comprimento variável no mesmo batch (slide 118).
3. Múltiplas cabeças.

## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import os
import random
import re

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook, tqdm
from typing import List

In [ ]:
# # Check which GPU we are using
# !nvidia-smi

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

# Carregamento do dataset 

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzfv aclImdb.tgz

## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
random.shuffle(x_train)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
x_train = x_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x in x_train[:3]:
    print(x[:100])

print('3 últimas amostras treino:')
for x in x_train[-3:]:
    print(x[:100])

print('3 primeiras amostras validação:')
for x in x_valid[:3]:
    print(x[:100])

print('3 últimas amostras validação:')
for x in x_valid[-3:]:
    print(x[:100])

random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

### Definindo funções de manipulação de texto.

In [ ]:
from typing import List

import re
import string


def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """

    return re.findall(r"[\w']+|[.,!?;]", text.lower())


from collections import Counter


def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """

    tokens = []

    for t in texts:
        tokens.extend(tokenize(t))

    return dict(Counter(tokens).most_common(max_tokens))

def items_em_comum(amostra, corpus):
    c = [value for value in amostra if value in corpus]
    return c


def concatenate_list_of_str(texts: List[str]):
    return ".".join(texts)

class Tokenizador():
    def __init__(self, texts, tokenizer=tokenize, vocab_size=3000):
        super().__init__()

        self.tokenizer = tokenizer
        self.vocab_size = vocab_size

        self.every_text = concatenate_list_of_str(texts)
        self.tokens_ocurrences = create_vocab(tokenize(self.every_text), max_tokens=vocab_size)

        self.lista_do_vocabulario = list(self.tokens_ocurrences.keys())

        self.dicionario_tokens = dict(zip(self.lista_do_vocabulario, range(len(self.lista_do_vocabulario))))

    def __call__(self, text):
        tokens = self.tokenizer(text)

        # Pegamos somente os tokens que existem no vocabulario
        tokens_keys = list(items_em_comum(tokens, self.lista_do_vocabulario))

        return list(itemgetter(*tokens_keys)(self.dicionario_tokens))

    def decode(self, ids):
        reversed_dict = dict(zip(self.dicionario_tokens.values(), self.dicionario_tokens.keys()))
        return list(itemgetter(*ids)(reversed_dict))


In [ ]:
### Criando classe do dataset

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int, vocab_size=1000):
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            print("Carregando dataset preprocessado")
        except Exception as e:
            # print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenizer(text)
                for i in range(len(tokens_key)-context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+context_size])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long()

Testando Dataset

In [ ]:
vocab_size = 3000

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=Tokenizador(dummy_texts, tokenize, vocab_size), context_size=3, vocab_size=vocab_size)
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False)

assert len(dummy_dataset) == 4
print('Passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[1, 2, 0],
     [5, 6, 7]])

correct_first_batch_target = torch.LongTensor([3,   0, ])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Dados de treino, validação e teste

In [ ]:
# Load datasets
context_size = 9

tokenizer = Tokenizador(x_train, tokenize, vocab_size=3000)

training_dataset = MyDataset(texts=x_train, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=x_valid, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=x_test, tokenizer=tokenizer, context_size=context_size)

print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

In [ ]:
class AttentionLayer(torch.nn.Module):

    def __init__(self, embedding_dim: int, max_seq_length: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
        """
        super().__init__()
        self.context_size = max_seq_length
        self.embedding_dim = embedding_dim

        # hidden_size for MLP
        hidden_size = 512

        # Linear projections
        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)
        self.w_0 = nn.Linear(embedding_dim, embedding_dim)

        # output MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.LeakyReLU(),
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, embedding_dim)
        )

        self.activation = nn.LeakyReLU()

        # cast to probabilities
        self.softmax = nn.Softmax(dim=-1)

        self.norm1 = nn.LayerNorm(embedding_dim)
        self.norm2 = nn.LayerNorm(embedding_dim)

        # Matriz triangular de mascara, convertida para Booleano
        # Onde vale 1, o valor deve ser substituida por um valor negativo alto no tensor de scores.
        self.causal_mask = torch.ones((max_seq_length, max_seq_length), device=device).triu(diagonal=1) == 1.0

    def forward(self, x_embeddings):

        k = self.w_k(x_embeddings)
        v = self.w_v(x_embeddings)
        q = self.w_q(x_embeddings)

        scores = torch.matmul(q, k.transpose(1, 2))

        # Onde a mascara vale 1, retornamos um valor negativo grande.
        # Onde a mascara vale zero, matemos intacto.
        probabilities = self.softmax(scores.masked_fill(self.causal_mask, -1e4))

        e = self.w_0(self.norm1(x_embeddings + torch.matmul(probabilities, v)))

        logits = self.mlp(e)

        return self.norm2(self.activation(logits + e))

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int,):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
        """

        super().__init__()
        embedding_dim = dim
        context_size = max_seq_length
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # hidden_size for MLP
        hidden_size = 512

        # tokens (words indexes) embedding and positional embedding
        self.c_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.p_embedding = nn.Embedding(vocab_size, embedding_dim)

        self.attention = nn.Sequential(*[AttentionLayer(embedding_dim=embedding_dim, max_seq_length=max_seq_length) for i in range(n_layers)])

        self.linear_output = nn.Linear(embedding_dim, vocab_size)

        # cast to probabilities
        self.softmax = nn.Softmax(dim=-1)

        self.positional_indexes = torch.arange(self.context_size, device=device).view(1, -1)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)

        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """


        input_embeddings = self.c_embedding(inputs)

        positional_embeddings = self.p_embedding(self.positional_indexes.repeat(inputs.shape[0], 1))

        x_embeddings = positional_embeddings + input_embeddings

        logits = self.attention(x_embeddings)

        return self.linear_output(logits)[:, -1, :]

Testando se o modelo processa os dados corretamente

In [ ]:
model = LanguageModel(
    vocab_size=vocab_size,
    max_seq_length=context_size,
    dim=32,
    n_layers=1,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

Verificando a perplexidade

In [ ]:
def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """


    crossentropy =  nn.functional.cross_entropy(logits,target)

    return crossentropy.exp()


n_examples = 100

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

print("logits shape: ", logits.shape)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {vocab_size}')

assert math.isclose(my_perplexity, vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

TREINAMENTO

In [ ]:
from queue import Queue
from threading import Thread


class DataManager(Thread):
    def __init__(self, data_loader, buffer_size=3, device=torch.device("cpu"), data_type=torch.float32):
        """
This manager intends to load a PyTorch dataloader like from disk into memory,
reducing the acess time. It does not easily overflow memory, because we set a
buffer size limiting how many samples will be loaded at once. Everytime a sample
is consumed by the calling thread, another one is replaced in the
buffer (unless we reach the end of dataloader).

A manger may be called exactly like a dataloader, an it's based in an internal
thread that loads samples into memory in parallel. This is specially useful
when you are training in GPU and processor is almost idle.

        :param data_loader: Base dataloader to load in parallel.
        :param buffer_size: How many samples to keep loaded (caution to not overflow RAM). Must be integer > 0. Default: 3.
        :param device: Torch device to put samples in, like torch.device("cpu") (default). It saves time by transfering in parallel.
        :param data_type: Automatically casts tensor type. Default: torch.float32.
        """
        super().__init__()
        self.buffer_queue = Queue(maxsize=buffer_size)
        self.data_loader = data_loader
        self.buffer_size = buffer_size
        self.device = device
        self.data_type = data_type

        self.dataloader_finished = False

    def run(self):
        """
Runs the internal thread that iterates over the dataloader until fulfilling the
buffer or the end of samples.
        """
        for i, sample in enumerate(self.data_loader):
            # Important to set before put in queue to avoid race condition
            # would happen if trying to get() in next() method before setting this flag
            if i >= len(self) - 1:
                self.dataloader_finished = True

            self.buffer_queue.put([x.to(dtype=self.data_type, device=self.device, non_blocking=True) for x in sample])

    def __iter__(self):
        """
Returns an iterable of itself.

        :return: Iterable around this class.
        """
        self.start()
        self.dataloader_finished = False
        return self

    def __next__(self):
        """
Intended to be used as iterator.

        :return: Next iteration element.
        """
        if self.dataloader_finished is True and self.buffer_queue.empty():
            raise StopIteration()

        return self.buffer_queue.get()

    def __len__(self):
        return len(self.data_loader)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast

max_examples = 60_000_000
eval_every_steps = 5000
lr = 4e-5
use_amp = True

model = LanguageModel(
    vocab_size=vocab_size,
    max_seq_length=context_size,
    dim=64,
    n_layers=2,
).to(device)
train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, num_workers=4)
validation_loader = DataLoader(valid_dataset, batch_size=1024, num_workers=4, )

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.9, min_lr=3e-5, patience=15000, threshold=1e-1, verbose=True)
scaler=GradScaler()


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    with autocast(enabled=use_amp):
        logits = model(input_ids)
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, )
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    with autocast(enabled=use_amp):
        logits = model(input_ids)
        logits = logits.reshape(-1, logits.shape[-1])
        target_ids = target_ids.reshape(-1)
        loss = nn.functional.cross_entropy(logits, target_ids,)
    return loss.item()

best_validation_ppl = 9999
train_losses = []
n_examples = 0
step = 0
pbar = tqdm(total=max_examples)
while n_examples < max_examples:
    for train_input_ids, train_target_ids in DataManager(train_loader, device=device, buffer_size=1, data_type=None):
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device))
        train_losses.append(loss)

        # LR scheduler
        scheduler.step(loss)

        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in DataManager(validation_loader, device=device, buffer_size=1, data_type=None)]))
                # Checkpoint to best models found.
                if best_validation_ppl > valid_ppl:
                    # Update the new best perplexity.
                    best_validation_ppl = valid_ppl
                    model.eval()
                    torch.save(model, "best_model.pth")

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        pbar.update(len(train_input_ids))
        if n_examples >= max_examples:
            break

pbar.close()

# Restore best model (checkpoint) found
model = torch.load("best_model.pth")

Avaliação no dataset de Teste

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in tqdm(test_loader)
    ]))

print(f'test perplexity: {test_ppl}')

Testando gerar uma sentença

In [ ]:
prompt = 'Frankenstein tells the story of gifted scientist Victor Frankenstein who succeeds in giving life to '
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenizer(text=prompt)
    input_ids_truncated = input_ids[-context_size:]# Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    predicted_word = tokenizer.decode([predicted_id,])
    prompt += predicted_word[0]
    print(prompt)